# Day 12

In [1]:
from itertools import combinations, product
from pathlib import Path

import networkx as nx
import numpy as np

In [2]:
input = Path("example.txt").read_text()
print(input)

RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE



In [3]:
offset = ord("A")
arr = np.array(
    [[ord(character) - offset for character in line] for line in input.splitlines()]
)
print(arr)

[[17 17 17 17  8  8  2  2  5  5]
 [17 17 17 17  8  8  2  2  2  5]
 [21 21 17 17 17  2  2  5  5  5]
 [21 21 17  2  2  2  9  5  5  5]
 [21 21 21 21  2  9  9  2  5  4]
 [21 21  8 21  2  2  9  9  4  4]
 [21 21  8  8  8  2  9  9  4  4]
 [12  8  8  8  8  8  9  9  4  4]
 [12  8  8  8 18  8  9  4  4  4]
 [12 12 12  8 18 18  9  4  4  4]]


In [4]:
PAD_VALUE = -2
pad_arr = np.pad(arr, pad_width=1, constant_values=PAD_VALUE)
print(pad_arr)

[[-2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2]
 [-2 17 17 17 17  8  8  2  2  5  5 -2]
 [-2 17 17 17 17  8  8  2  2  2  5 -2]
 [-2 21 21 17 17 17  2  2  5  5  5 -2]
 [-2 21 21 17  2  2  2  9  5  5  5 -2]
 [-2 21 21 21 21  2  9  9  2  5  4 -2]
 [-2 21 21  8 21  2  2  9  9  4  4 -2]
 [-2 21 21  8  8  8  2  9  9  4  4 -2]
 [-2 12  8  8  8  8  8  9  9  4  4 -2]
 [-2 12  8  8  8 18  8  9  4  4  4 -2]
 [-2 12 12 12  8 18 18  9  4  4  4 -2]
 [-2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2]]


In [5]:
G = nx.Graph()

for height, width in product(range(arr.shape[0]), range(arr.shape[1])):
    G.add_node((height, width))

shifts = [1]
axes = [0, 1]

for shift, axis in product(shifts, axes):
    same_crop = (pad_arr == np.roll(pad_arr, shift=shift, axis=axis)) & (
        pad_arr != PAD_VALUE
    )
    node_1 = np.argwhere(same_crop) - 1
    direction = shift * np.array([1 if axis == 0 else 0, 1 if axis == 1 else 0])
    node_2 = node_1 - direction

    for n1, n2 in zip(node_1, node_2):
        G.add_edge(tuple(n1), tuple(n2))
print(len(G.nodes))

100


## Part I

In [6]:
cost = 0
for component in nx.components.connected_components(G):
    perimeter = 0
    area = 0
    for node in component:
        perimeter += 4 - len(G.edges(node))
        area += 1
    cost += perimeter * area
cost

1930

## Part II

In [9]:
def is_line(node_1, node_2) -> bool:

    return node_1[0] == node_2[0] or node_1[1] == node_2[1]


def is_concave(G, connector, node_1, node_2) -> bool:

    return set(nx.neighbors(G, node_1)).intersection(
        set(nx.neighbors(G, node_2))
    ) == set([connector])

In [8]:
costs = []
for component in nx.components.connected_components(G):
    sides = 0
    area = len(component)
    for node in component:
        neighbors = list(nx.neighbors(G, node))
        num_neighbors = len(neighbors)
        if num_neighbors == 0:
            # Single block
            sides += 4
        elif num_neighbors == 1:
            # Tip
            sides += 2
        elif num_neighbors == 2:
            # Corner (convex) or elbow (convex + concave)
            n1, n2 = neighbors
            if is_line(n1, n2):
                continue
            if is_concave(G, node, n1, n2):
                sides += 1
            sides += 1
        else:
            # Check for concave corners
            for n1, n2 in combinations(neighbors, 2):
                if is_line(n1, n2):
                    continue
                if is_concave(G, node, n1, n2):
                    sides += 1
    costs.append(sides * area)
sum(costs)

1206